**Assignment 1: Founder-Investor Matching AI Model**

In [2]:
import pandas as pd
import numpy as np
import google.generativeai as genai 

In [ ]:
genai.configure(api_key="AIzaSyBzXU3SFo4gOLs-QF9vxjk######")  # I have not provided full API key

# Load the dataset
file_path = "Combined file.xlsx - Sheet1.json"
df = pd.read_json(file_path)

# Sample founder input
founder_profile = {
    "Industry": "Fintech",
    "Startup_Stage": "Seed",
    "Funding_Required": 500000  # In USD
}

In [ ]:
import google.generativeai as genai

genai.configure(api_key="AIzaSyBzXU3SFo4gOLs-########")

models = genai.list_models()
for model in models:
    print(model.name, model.supported_generation_methods)


models/chat-bison-001 ['generateMessage', 'countMessageTokens']
models/text-bison-001 ['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-001-tuning ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCac

In [5]:
def preprocess_cheque_range(cheque_range):
    """Extract min and max funding range from cheque_range."""
    try:
        amounts = cheque_range.replace("$", "").replace("M", "000000").replace("K", "000").split(" - ")
        return int(amounts[0]), int(amounts[1]) if len(amounts) > 1 else int(amounts[0])
    except:
        return 0, 0

# Process cheque range
df[['Min_Cheque', 'Max_Cheque']] = df['Cheque_range'].apply(lambda x: pd.Series(preprocess_cheque_range(x)))



In [8]:
def calculate_match_score(investor, founder):
    """Calculate a compatibility score between a founder and an investor."""
    score = 0
    
    # Industry match
    if founder['Industry'] in investor['Industry']:
        score += 40
    
    # Stage match
    if founder['Startup_Stage'] in investor['Stage']:
        score += 30
    
    # Funding match
    if investor['Min_Cheque'] <= founder['Funding_Required'] <= investor['Max_Cheque']:
        score += 30
    
    return score

df['Match_Score'] = df.apply(lambda x: calculate_match_score(x, founder_profile), axis=1)

# Rank investors
df_sorted = df.sort_values(by='Match_Score', ascending=False)

# Select top matches
top_investors = df_sorted[['Name', 'Website', 'Industry', 'Stage', 'Cheque_range', 'Match_Score']].head(10)

# ✅ **Integrate Gemini API**
model = genai.GenerativeModel("gemini-1.5-pro")  # ✅ Correct Model
prompt = f"Give insights on the best funding strategies for a {founder_profile['Industry']} startup at {founder_profile['Startup_Stage']} stage requiring ${founder_profile['Funding_Required']}."

response = model.generate_content(prompt)

# Output results
print(top_investors.to_string(index=False))
print("\n🔹 Gemini AI Insights:\n", response.text)

                  Name                               Website                                                                                                                                                                                             Industry                                             Stage      Cheque_range  Match_Score
              Q Angels                   https://qangels.org                                                                                                                                                                                        SaaS, Fintech               Pre-seed, Idea, Prototype/MVP, Seed      $50K - $500K          100
        Camel Ventures           https://www.camel.ventures/                                                                                                                                                                                              Fintech      Pre-seed, Idea, Prototype/MVP, Seed, Series+       $100K - 